# MAST30034 Applied Data Science Project 2

## Geospatial Analysis

In [1]:
from pyspark.sql import SparkSession, functions as F, DataFrame
from pyspark.sql.functions import col
import geopandas as gpd
import folium
import pandas as pd

In [2]:
spark = (
    SparkSession.builder.appName("Geospatial_Analysis")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/09/12 10:40:13 WARN Utils: Your hostname, Yis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.16.32.216 instead (on interface en0)
22/09/12 10:40:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/12 10:40:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/12 10:40:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/12 10:40:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/12 10:40:17 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/09/12 10:40:17 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
data = spark.read.parquet('../data/curated/outlier_data.parquet')
sf = gpd.read_file("../data/external/shapefile/SA2_2021_AUST_GDA2020.shp")

In [5]:
data.limit(5)

SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,dollar_value,order_id,order_datetime,name,tags,take rate,revenue level,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Completed_Year_12,Did_Not_Attend_School,TOT_P_P,Age_0_4_yr_P,Age_5_14_yr_P,Age_15_19_yr_P,Age_20_24_yr_P,Age_25_34_yr_P,Age_35_44_yr_P,Age_45_54_yr_P,Age_55_64_yr_P,Age_65_74_yr_P,Age_75_84_yr_P,Age_85ov_P,state
null,6915,3067,7470,47086412084,8.195033092624405,3b7b90dd-68a6-4f6...,2021-08-21,Eros Limited,[digital goods: b...,2.52,c,Kenneth Tran,480 Blanchard Land,Male,Western Austraia,836.8382313624497,235.6218149377005,1437.341332794222,42.58467505548575,1530.1537458327882,2.439455031966128,2478.7919578594638,28.059934734053822,6548.471659656898,null,813.4840358738074,335.69086501383237,332.347258834665,855.0453756447644,877.1825413814199,884.6771463039821,897.2375582660876,717.5282143742763,350.91326275555485,110.15571530693009,WA
null,6915,3067,7470,52065114842,106.91043190212513,49d4d2de-6783-435...,2021-08-14,Gravida Non PC,[digital goods: b...,3.98,b,Kenneth Tran,480 Blanchard Land,Male,Western Austraia,836.8382313624497,235.6218149377005,1437.341332794222,42.58467505548575,1530.1537458327882,2.439455031966128,2478.7919578594638,28.059934734053822,6548.471659656898,null,813.4840358738074,335.69086501383237,332.347258834665,855.0453756447644,877.1825413814199,884.6771463039821,897.2375582660876,717.5282143742763,350.91326275555485,110.15571530693009,WA
null,6915,3067,7470,13118172970,65.98639083026582,267b6bc3-3977-49d...,2021-08-15,Placerat Cras Inc...,"[gift, card, no...",4.57,b,Kenneth Tran,480 Blanchard Land,Male,Western Austraia,836.8382313624497,235.6218149377005,1437.341332794222,42.58467505548575,1530.1537458327882,2.439455031966128,2478.7919578594638,28.059934734053822,6548.471659656898,null,813.4840358738074,335.69086501383237,332.347258834665,855.0453756447644,877.1825413814199,884.6771463039821,897.2375582660876,717.5282143742763,350.91326275555485,110.15571530693009,WA
null,6915,3067,7470,35223308778,96.84997524124665,2f92ee68-38a7-4f1...,2021-07-15,Euismod In Corp.,"[books, periodic...",4.19,b,Kenneth Tran,480 Blanchard Land,Male,Western Austraia,836.8382313624497,235.6218149377005,1437.341332794222,42.58467505548575,1530.1537458327882,2.439455031966128,2478.7919578594638,28.059934734053822,6548.471659656898,null,813.4840358738074,335.69086501383237,332.347258834665,855.0453756447644,877.1825413814199,884.6771463039821,897.2375582660876,717.5282143742763,350.91326275555485,110.15571530693009,WA
null,6915,3067,7470,58392414752,30.000483014621604,f82dac78-e0ec-4da...,2021-08-27,Mattis Ornare Lec...,[florists supplie...,1.47,c,Kenneth Tran,480 Blanchard Land,Male,Western Austraia,836.8382313624497,235.6218149377005,1437.341332794222,42.58467505548575,1530.1537458327882,2.439455031966128,2478.7919578594638,28.059934734053822,6548.471659656898,null,813.4840358738074,335.69086501383237,332.347258834665,855.0453756447644,877.1825413814199,884.6771463039821,897.2375582660876,717.5282143742763,350.91326275555485,110.15571530693009,WA


In [7]:
data=data.withColumn("take_rate*dollar_value", F.col("take rate")*F.col("dollar_value"))

In [8]:
SA2_data = data \
            .groupBy("SA2_CODE_2021") \
            .agg(
                F.mean("dollar_value").alias("mean_amount"),
                F.sum("dollar_value").alias("total_amount"),
                F.count("order_id").alias("order_volume"),
                F.mean("Median_tot_hhd_inc_weekly").alias("household_income"),
                F.mean("take_rate*dollar_value").alias("mean_take_rate*dollar_value"),
            ) \
            .orderBy("SA2_CODE_2021") \
            .toPandas()

In [46]:
sf

,SA2_CODE21,geometry
0,101021007,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."
2,101021009,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3..."
...,...,...
2468,901031003,"MULTIPOLYGON (((150.69567 -35.18295, 150.69556..."
2469,901041004,"MULTIPOLYGON (((167.96325 -29.07212, 167.96326..."
2470,997979799,None
2471,999999499,None


In [47]:
# Code from MAST30034 Applied Data Science Tutorial 2
# Convert the geometry shape to to latitude and longitude
sf = sf[['SA2_CODE21','geometry']]
sf['geometry'] = sf['geometry'] \
                    .to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
gdf = gpd.GeoDataFrame(sf)
geoJSON = gdf[['SA2_CODE21', 'geometry']].to_json()

In [49]:
SA2_data['order_volume'].max()

781493

In [50]:
m1 = folium.Map(location=[-33.8, 151.2],
                  tiles="cartodbpositron", zoom_start=4)

c1 = folium.Choropleth(
    geo_data=geoJSON,
    name='Income By SA2',
    data=SA2_data.reset_index(),
    columns=['SA2_CODE_2021', 'household_income'],
    key_on='properties.SA2_CODE21',
    fill_color='YlOrRd',
    nan_fill_color='black',
    legend_name='Income By SA2'
)

c1.add_to(m1)

m1.save("../plots/income.html")

In [51]:
m2 = folium.Map(location=[-33.8, 151.2],
                  tiles="cartodbpositron", zoom_start=4)

c2 = folium.Choropleth(
    geo_data=geoJSON,
    name='Transaction Volume By SA2',
    data=SA2_data.reset_index(),
    columns=['SA2_CODE_2021', 'order_volume'],
    key_on='properties.SA2_CODE21',
    fill_color='YlOrRd',
    nan_fill_color='black',
    legend_name='Transaction Volume By SA2'
)

c2.add_to(m2)

m2.save("../plots/volume.html")

In [52]:
m3 = folium.Map(location=[-33.8, 151.2],
                  tiles="cartodbpositron", zoom_start=4)

c3 = folium.Choropleth(
    geo_data=geoJSON,
    name='Average Transaction Amount By SA2',
    data=SA2_data.reset_index(),
    columns=['SA2_CODE_2021', 'mean_amount'],
    key_on='properties.SA2_CODE21',
    fill_color='YlOrRd',
    nan_fill_color='black',
    legend_name='Average Transaction Amount By SA2'
)

c3.add_to(m3)

m3.save("../plots/avg_amount.html")

In [53]:
m4 = folium.Map(location=[-33.8, 151.2],
                  tiles="cartodbpositron", zoom_start=4)

c4 = folium.Choropleth(
    geo_data=geoJSON,
    name='Average Take Rate * Dollar Value By SA2',
    data=SA2_data.reset_index(),
    columns=['SA2_CODE_2021', 'mean_amount'],
    key_on='properties.SA2_CODE21',
    fill_color='YlOrRd',
    nan_fill_color='black',
    legend_name='Average Take Rate * Dollar Value By SA2'
)

c4.add_to(m4)

m4.save("../plots/avg_take_rate_dollar_val.html")